### Инструмент для анализа текста

**Основная задача**: Создать инструмент для анализа текста, который обрабатывает текстовые записи (например, отзывы пользователей, статьи или отрывки из книг) и предоставляет некоторые выводы, статистику и преобразования. После этого применить инструмент для анализа текста.

In [19]:
TEXT = """**_Stellar Blade_** is the latest exclusive to hit the PlayStation 5 that excels
at delivering a highly riveting action RPG. Shift Up Corporation has crafted an
impressive linear title that takes influences from some of the most renowned
games in recent times. **Even though the parallels are quite apparent, _Stellar
Blade_ doesn’t try to reinvent the wheel but rather refines what players are
already familiar with.**  _Stellar Blade_ is centered around EVE, a member of
the 7th Airborne Division w..."""

## Часть 1: Сбор и предварительная обработка данных

Написать функцию `preprocess_text`, которая принимает на вход текст и очищает его. Функция должна:
   - Преобразовывать текст в строчные буквы.
   - Удалять ненужные знаки препинания и специальные символы, но сохранять знаки препинания в конце предложения (`.` `!` `?`).
   - Удалить лишние пробелы между словами.
   - Вернуть очищенный текст.


In [20]:
import re
def preprocess_text(text):
    text = text.lower().strip()
    text = re.sub(r'[^a-zA-Z0-9\s.!?]', '', text) # del all trash symbols
    text = re.sub(r'\s+', ' ', text) # del all trash bspaces
    text = re.sub(r'\s([.!?])', r'\1', text) # del bsaces before !?.
    text = re.sub(r'([.!?]){2,}', r'\1', text) # del acsess !?.
    return text

input_texts = [
    "Hello, World!\n   This is a test...",
    "Python is fun!!! #coding",
    "  Spaces should be    removed."
]

processed_texts = [preprocess_text(text) for text in input_texts]
print(processed_texts,end='\n')

['hello world! this is a test.', 'python is fun! coding', 'spaces should be removed.']


## Часть 2: Анализ частоты слов

Написать функцию `word_frequency`, которая берет очищенный текст (из части 1) и возвращает словарь, где ключами являются слова, а значениями - их частота в тексте. Функция должна:
   - Подсчитать, как часто каждое слово встречается в тексте.
   - Игнорировать любые общие стоп-слова. Вы можете использовать предопределенный список, приведенный ниже, или использовать свой собственный. Подумайте о том, чтобы сделать его настраиваемым.

**Предопределённые стоп-слова**:
```python
stop_words = ["and", "the", "is", "in", "it", "you", "that", "to", "of", "a", "with", "for", "on", "this", "at", "by", "an"]
```

In [21]:
def word_frequency(text, stop_words = None):
    if stop_words is None:
        stop_words = ["i", "and", "the", "is", "in", "it", "you", "that", "to", "of", 
                     "a", "with", "for", "on", "this", "at", "by", "an", "be", "should"]
    words = text.lower().split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    clean_words = [word.strip('.!?,') for word in filtered_words]
    freq_dict = {}
    for word in clean_words:
        freq_dict[word] = freq_dict.get(word, 0) + 1
    return freq_dict

## Часть 3: Извлечение информации

Написать функцию `extract_information`, которая берет неочищенный текст (**не из части 1**) и извлекает определенные типы информации на основе настраиваемых шаблонов regex, предоставленных в качестве keyword-аргументов. Функция должна возвращать словарь, в котором ключи - это типы совпадений, а значения - списки найденных совпадений.

**Поддерживаемые типы информации**:
1. Адреса электронных почт
2. Телефонные номера
3. Даты
4. Время
5. Цены
6. Дополнительные данные по желанию пользователя

**Инструкции**:
- Функция должна принимать следующие keyword-only аргументы с шаблонами regex по умолчанию:
  - `email_pattern` (default: `r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'`)
  - `phone_pattern` (default: `r'\+?\d[\d -]{8,12}\d'`)
  - `date_pattern` (default: `r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b'`)
  - `time_pattern` (default: `r'\b\d{1,2}:\d{2}\s*[APap][mM]?\b'`)
  - `price_pattern` (по умолчанию: `r'\$\d+(?:\.\d{2})?'`)
  - `**extra_patterns`: Дополнительные аргументы для пользовательских шаблонов.

- Функция должна:
  - Найти все совпадения в по каждому шаблону во всём тексте.
  - Хранить совпадения в словаре результатов под соответствующим типом.
  - Возвращать словарь со всеми найденными совпадениями.


In [22]:
import re
def extract_information(text, *,
                        email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
                        phone_pattern = r'\+?\d?\(?\d{3}\)?[\d -]{8,12}\d',
                        date_pattern = r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b',
                        time_pattern = r'\b\d{1,2}:\d{2}\s*[APap][mM]?\b',
                        price_pattern = r'\$\d+(?:\.\d{2})?', 
                        **patterns):
    default_patterns = {'emails': re.findall(email_pattern, text),
        'phone_numbers': re.findall(phone_pattern, text),
        'dates': re.findall(date_pattern, text),
        'times': re.findall(time_pattern, text),
        'prices': re.findall(price_pattern, text)}
    for name, pattern in patterns.items():
        default_patterns[name] = re.findall(pattern, text)
    return default_patterns

## Часть 4: Анализ настроения

Написать функцию `analyze_sentiment`, которая берет очищенный текст (из задания 1) и анализирует его настроение на основе заранее определенных положительных и отрицательных слов. Функция должна возвращать оценку настроения текста.

**Предназначенные списки слов**:
- **Положительные слова** (по умолчанию):
```python
positive_words = ["good", "great", "happy", "joy", "excellent", "fantastic", "love", "best"]
```
- **Негативные слова** (по умолчанию):
```python
negative_words = ["bad", "sad", "hate", "terrible", "awful", "poor", "worst"]
```

- Функция должна принимать следующие keyword-аргументы:
  - `positive_words`: Список положительных слов (по умолчанию, как показано выше).
  - `negative_words`: Список отрицательных слов (по умолчанию, как показано выше).

- Функция должна:
  - Подсчитать количество положительных и отрицательных слов в каждой записи текста.
  - Рассчитать балл настроения для каждой записи:
    - Sentiment Score = (Number of Positive Words - Number of Negative Words)
  - Вернуть оценку настроения текста.

In [23]:
def analyze_sentiment(text, positive_words=None, negative_words=None):
    if positive_words is None:
        positive_words = ["good", "great", "happy", "joy", "excellent", 
                         "fantastic", "love", "best", "amazing", "fun"]
    if negative_words is None:
        negative_words = ["bad", "sad", "hate", "terrible", "awful", 
                         "poor", "worst"]
    cleaned_words = [word.strip('.!?').lower() for word in text.split()]
    positive_count = sum(cleaned_words.count(word) for word in positive_words)
    negative_count = sum(cleaned_words.count(word) for word in negative_words)
    sentiment_score = positive_count - negative_count 
    return sentiment_score

## Часть 5: Обобщение текста

Написать функцию `summarize_text`, которая берет очищенный текст (из части 1) и обобщает его, извлекая наиболее важные предложения на основе их релевантности. Функция должна позволять пользователю указывать в качестве параметра желаемый коэффициент сжатия.

- Функция должна принимать следующие параметры:
  - `text`: Одиночный очищенный текст, который нужно обобщить.
  - `compression_ratio`: Число от 0 до 1, представляющее желаемое сжатие (например, 0,6 для 60-процентного обобщения).
  - `min_threshold`: Целое число, указывающее минимальное количество оставшихся предложений (по умолчанию 2).

- Функция должна:
  - Разделить текст на предложения.
  - Ранжировать предложения на основе их важности (можно использовать простые метрики, такие как длина, частота слов, наличие значимых слов или любые другие метрики, которые вы считаете наиболее подходящими).
  - Используйте рекурсивный подход для динамического отбора предложений с наивысшим рейтингом, удаляя каждое выбранное предложение и заново ранжируя оставшиеся, пока не будет достигнут желаемый коэффициент сжатия.
  - Верните обобщенный текст в виде строки, состоящей из выбранных предложений.


In [24]:
import re

def summarize_text(text, compression_ratio=0.6, min_threshold=2):
    if not text or not text.strip():
        return ""
    sentences = [s.strip() for s in re.split(r'[.!?]', text) if s.strip()]
    target_length = max(min_threshold, int(len(sentences) * (1 - compression_ratio)))
    if len(sentences) <= target_length:
        return '. '.join(sentences) + '.'
    
    def rank_sentences(sentences):
        word_frequencies = {}
        for sentence in sentences:
            for word in sentence.split():
                word_lower = word.lower()
                if word_lower not in word_frequencies:
                    word_frequencies[word_lower] = 0
                word_frequencies[word_lower] += 1
        sentence_scores = {}
        for sentence in sentences:
            words = sentence.split()
            sentence_scores[sentence] = sum(word_frequencies[word.lower()] for word in words)
        return sentence_scores
    
    def recursive_summarization(sentences, target_length):
        if len(sentences) <= target_length:
            return sentences
        sentence_scores = rank_sentences(sentences)
        highest_ranked = max(sentence_scores, key=sentence_scores.get)
        reduced_sentences = [s for s in sentences if s != highest_ranked]
        return recursive_summarization(reduced_sentences, target_length)
    
    original_sentences = sentences[:]
    summarized_sentences = recursive_summarization(sentences, target_length)
    summarized_sentences.sort(key=lambda sentence: original_sentences.index(sentence))

    return '. '.join(summarized_sentences) + '.'


t = '''**_Stellar Blade_** is the latest exclusive to hit the PlayStation 5 that excels
at delivering a highly riveting action RPG. Shift Up Corporation has crafted an
impressive linear title that takes influences from some of the most renowned
games in recent times. **Even though the parallels are quite apparent, _Stellar
Blade_ doesn’t try to reinvent the wheel but rather refines what players are
already familiar with.**  _Stellar Blade_ is centered around EVE, a member of
the 7th Airborne Division w...'''

test_text = preprocess_text(t)
print(test_text)
summary = summarize_text((test_text))
print("Summary:")
print(summary)

stellar blade is the latest exclusive to hit the playstation 5 that excels at delivering a highly riveting action rpg. shift up corporation has crafted an impressive linear title that takes influences from some of the most renowned games in recent times. even though the parallels are quite apparent stellar blade doesnt try to reinvent the wheel but rather refines what players are already familiar with. stellar blade is centered around eve a member of the 7th airborne division w.
Summary:
shift up corporation has crafted an impressive linear title that takes influences from some of the most renowned games in recent times. stellar blade is centered around eve a member of the 7th airborne division w.


## Часть 6: Визуализация частоты слов

Написать функцию `visualize_word_frequency`, которая берет словарь частот слов (полученный в части 2) и визуализирует частоты в текстовом формате.

- Функция должна принимать словарь, в котором ключами являются слова, а значениями - их частоты.
- На выходе должно отображаться каждое слово, за которым следует горизонтальная полоса, представляющая его частоту. Для создания полос используйте простой символ (например, `*`).
- Длину каждой полосы можно масштабировать в зависимости от максимальной частоты, чтобы обеспечить читабельность.
- Функция должна иметь параметр `max_threshold`, чтобы ограничить количество строк в выводе (по умолчанию 20).
- Вывод должен быть отсортирован от наиболее до наименее часто используемых слов

In [25]:
def visualize_word_frequency(word_frequency, max_threshold=20):
    sorted_words = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)[:max_threshold]
    for word, value in sorted_words:
        print(f'{word}: {value * "*"}')

## Часть 7: Функции высшего порядка для анализа текста

Написать функцию `apply_analysis`, которая принимает список функций анализа и одну текстовую запись. Функция должна применять каждую функцию анализа к текстовой записи и возвращать словарь результатов.

- Функция должна принимать следующие параметры:
  - `text`: Один очищенный текст для анализа.
  - `analysis_functions`: Список функций для применения к тексту. Каждая функция должна принимать на вход один текст и возвращать результат (например, частоту слов, анализ настроения).

- Функция должна:
  - Перебирать список функций анализа, применяя каждую из них к текстовой записи и сохраняя результат в словаре под описательным ключом (например, именем функции).
  - Возвращать словарь, содержащий результаты всех примененных функций анализа.


In [26]:
def apply_analysis(text, functions):
    values = dict()
    for func in functions:
        values[func.__name__] = func(text)
    return values

## Часть 8: Обернуть всё в класс

Создайте класс `TextAnalyzer`, который инкапсулирует всю функциональность из предыдущих задач. Класс должен включать методы для каждого из следующих действий:

1. **Инициализация**:
   - Класс должен принимать один текст при инициализации, хранить исходный текст и автоматически создавать его предобработанную версию.

2. **Методы**:
   - `word_frequency`: Анализирует частоту слов из предварительно обработанного текста.
   - `extract_information`: Извлекает электронные письма, номера телефонов, даты, время и цены из предварительно обработанного текста.
   - `analyze_sentiment`: Выполняет анализ настроения предварительно обработанного текста.
   - `summarize_text`: Обобщает предварительно обработанный текст на основе коэффициента сжатия.
   - `visualize_word_frequency`: Визуализирует данные о частоте слов в тексте.
   - `apply_analysis`: Применяет список функций анализа к предварительно обработанному тексту.


- Класс должен сохранять все необходимые внутренние состояния, такие как исходный и обработанный текст.
- Каждый метод должен вызываться независимо, и класс должен позволять настраивать поведение там, где это возможно (например, передавать пользовательские списки слов или шаблоны regex).
- Убедитесь, что методы хорошо документированы и что класс может быть легко инстанцирован с вводом текста для анализа.


In [27]:
import re
from typing import List, Dict, Any
import heapq

class TextAnalyzer:
    def __init__(self, text: str):
        self.original_text = text
        self.cleaned_text = self.preprocess_text()

    def preprocess_text(self):
        text = self.original_text.lower().strip()
        text = re.sub(r'[^a-zA-Z0-9\s.!?]', '', text) # del all trash symbols
        text = re.sub(r'\s+', ' ', text) # del all trash bspaces
        text = re.sub(r'\s([.!?])', r'\1', text) # del bsaces before !?.
        text = re.sub(r'([.!?]){2,}', r'\1', text) # del acsess !?.
        return text

    def word_frequency(self, stop_words = ["i", "and", "the", "is", "in", "it", "you", "that", "to", "of", "a", "with", "for", "on", "this", "at", "by", "an", "be", "should"]) -> dict:
        words = self.cleaned_text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        clean_words = [word.strip('.!?,') for word in filtered_words]
        freq_dict = {}
        for word in clean_words:
            freq_dict[word] = freq_dict.get(word, 0) + 1
        return freq_dict

    def extract_information(self, 
                        email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
                        phone_pattern = r'\+?\d?\(?\d{3}\)?[\d -]{8,12}\d',
                        date_pattern = r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b',
                        time_pattern = r'\b\d{1,2}:\d{2}\s*[APap][mM]?\b',
                        price_pattern = r'\$\d+(?:\.\d{2})?', 
                        **patterns) -> dict:
        default_patterns = {'emails': re.findall(email_pattern, self.original_text),
            'phone_numbers': re.findall(phone_pattern, self.original_text),
            'dates': re.findall(date_pattern, self.original_text),
            'times': re.findall(time_pattern, self.original_text),
            'prices': re.findall(price_pattern, self.original_text)}
        for name, pattern in patterns.items():
            default_patterns[name] = re.findall(pattern, self.original_text)
        return default_patterns

    def analyze_sentiment(self, positive_words=["good", "great", "happy", "joy", "excellent", "fantastic", "love", "best", "amazing", "fun"],
                      negative_words=["bad", "sad", "hate", "terrible", "awful", "poor", "worst"]) -> int:
        cleaned_words = [word.strip('.!?').lower() for word in self.original_text.split()]
        positive_count = sum(cleaned_words.count(word) for word in positive_words)
        negative_count = sum(cleaned_words.count(word) for word in negative_words)
        sentiment_score = positive_count - negative_count 
        return sentiment_score

    def summarize_text(self, compression_ratio=0.6, min_threshold=2):
        text = self.cleaned_text
        if not text or not text.strip():
            return ""
        sentences = [s.strip() for s in re.split(r'[.!?]', text) if s.strip()]
        target_length = max(min_threshold, int(len(sentences) * (1 - compression_ratio)))
        if len(sentences) <= target_length:
            return '. '.join(sentences) + '.'
    
        def rank_sentences(sentences):
            word_frequencies = {}
            for sentence in sentences:
                for word in sentence.split():
                    word_lower = word.lower()
                    if word_lower not in word_frequencies:
                        word_frequencies[word_lower] = 0
                    word_frequencies[word_lower] += 1
            sentence_scores = {}
            for sentence in sentences:
                words = sentence.split()
                sentence_scores[sentence] = sum(word_frequencies[word.lower()] for word in words)
            return sentence_scores
    
        def recursive_summarization(sentences, target_length):
            if len(sentences) <= target_length:
                return sentences
            sentence_scores = rank_sentences(sentences)
            highest_ranked = max(sentence_scores, key=sentence_scores.get)
            reduced_sentences = [s for s in sentences if s != highest_ranked]
            return recursive_summarization(reduced_sentences, target_length)
    
        original_sentences = sentences[:]
        summarized_sentences = recursive_summarization(sentences, target_length)
        summarized_sentences.sort(key=lambda sentence: original_sentences.index(sentence))

        return '. '.join(summarized_sentences) + '.'

    def visualize_word_frequency(self, max_threshold=20):
        word_frequency = self.word_frequency()
        sorted_words = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)[:max_threshold]
        for word, value in sorted_words:
            print(f'{word}: {value * "*"}')

    def apply_analysis(self, functions) -> dict:
        text = self.original_text
        values = dict()
        for func in functions:
            values[func.__name__] = func(text)
        return values
    

## Часть 9: Анализ текста

Провести анализ текста. Воспользоваться разработанными инструментами, чтобы:

- Нормализовать текст
- Узнать наиболее часто встречающиеся слова
- Оценить настроение текста
- Определить основную информацию текста

Представить результаты оценок и сделайть вывод о содержании текста.

In [28]:
# final_boss = '**_Stellar Blade_** is the latest exclusive to hit the PlayStation 5 that excels at delivering a highly riveting action RPG. Shift Up Corporation has crafted an impressive linear title that takes influences from some of the most renowned games in recent times. **Even though the parallels are quite apparent, _Stellar Blade_ doesn’t try to reinvent the wheel but rather refines what players are already familiar with.**  _Stellar Blade_ is centered around EVE, a member of the 7th Airborne Division w...'

r = TextAnalyzer(TEXT)
print('normal text:\n', r.preprocess_text())
word_count = r.word_frequency()
for word, frequency in sorted(word_count.items(), key=lambda x: x[1], reverse=True):
    print(f'{word}: {frequency}')
print('mood text:\n', r.analyze_sentiment())
print('important info:\n', r.summarize_text(0.5))

normal text:
 stellar blade is the latest exclusive to hit the playstation 5 that excels at delivering a highly riveting action rpg. shift up corporation has crafted an impressive linear title that takes influences from some of the most renowned games in recent times. even though the parallels are quite apparent stellar blade doesnt try to reinvent the wheel but rather refines what players are already familiar with. stellar blade is centered around eve a member of the 7th airborne division w.
stellar: 3
blade: 3
are: 2
latest: 1
exclusive: 1
hit: 1
playstation: 1
5: 1
excels: 1
delivering: 1
highly: 1
riveting: 1
action: 1
rpg: 1
shift: 1
up: 1
corporation: 1
has: 1
crafted: 1
impressive: 1
linear: 1
title: 1
takes: 1
influences: 1
from: 1
some: 1
most: 1
renowned: 1
games: 1
recent: 1
times: 1
even: 1
though: 1
parallels: 1
quite: 1
apparent: 1
doesnt: 1
try: 1
reinvent: 1
wheel: 1
but: 1
rather: 1
refines: 1
what: 1
players: 1
already: 1
familiar: 1
with: 1
centered: 1
around: 1
eve:


### АНАЛИЗ РЕЗУЛЬТАТОВ:

Нормализация: текст очищен от спецсимволов, сохранена структура предложений, удалены лишние пробелы и форматирование

Частотный анализ:показана частоты слов, удалены стоп-слова

Тональность: текст средней эмоцианальности, кол-во хороших и плохих слов одинаковое

Ключевая информация:
Название игры: Stellar Blade
Платформа: PlayStation 5
Жанр: action RPG
Разработчик: Shift Up Corporation

Общий вывод: Текст представляет собой профессиональный обзор игры, написанный с позитивом. Автор считает игру качественным и оценил ее преемственность с другими сериями жанра. Текст информативен, содержит ключевые детали об игре
"""

### ***ОБЩИЙ ВЫВОД ПО РАБОТЕ:***
*В рамках выполнения задания была реализована программа для анализа текстов, соответствующая поставленной задаче. Основная цель заключалась в создании инструмента, который позволяет обработать текстовые данные и получить их ключевые характеристики.Работа позволила углубить мои знания в области анализа данных и программирования, а также приобрести практический опыт в создании аналитических инструментов.*